In [ ]:
import numpy as np
import copy
import pandas as pd
import os
import sys
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

n_cores = multiprocessing.cpu_count()


sys.path.insert(1, os.path.realpath(os.path.pardir))

In [ ]:
# # load the data


# # grn_filenames = ("../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_MALE-tumor_power10.h5",
# #                  "../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_FEMALE-tumor_power10.h5")

# grn_directory = "../data/networks/bonobo/"


# tissue = 'YEAST'
# target = 'sVSg2'
# network_inf_method = 'BONOBO'
# sex_and_gender = True
# gsea_parent_folder_name = 'BONOBO_yeast'
# save_gsea_results_notes = ''


# # do the data have index and header??
# # if it is .csv, most likely the sep = ","
# #  if it is .tsv, most likely the sep = "\t"
# index_col = 0
# header = 0
# sep = ','


# import os
# import re

# def get_filenames(directory):
#     filenames = []
#     pattern = re.compile(r'^bonobo_th01_s(0?[1-9]|1[0-3])\.h5$')
#     for filename in os.listdir(directory):
#         if os.path.isfile(os.path.join(directory, filename)) and pattern.match(filename):
#             filenames.append(grn_directory+filename)
#     return filenames

# # Example usage
# directory_path = grn_directory
# grn_filenames = get_filenames(directory_path)



# # load the gene regulatory networks into PANDAS dataframes and then to a list
# grns = []
# for grn_filename in grn_filenames:
#     if 'h5' not in grn_filename:
#         grn_pd = pd.read_csv(
#             grn_filename,  index_col=index_col, header=header, sep=sep)
#     else:
#         grn_pd = pd.read_hdf(grn_filename)

#     grns.append(grn_pd)

In [ ]:
# load the data


# grn_filenames = ("../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_MALE-tumor_power10.h5",
#                  "../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_FEMALE-tumor_power10.h5")

grn_directory = "../data/networks/bonobo/"


tissue = 'YEAST'
network_inf_method = 'BONOBO'
sex_and_gender = True
gsea_parent_folder_name = 'BONOBO_yeast_new'
save_gsea_results_notes = ''


# do the data have index and header??
# if it is .csv, most likely the sep = ","
#  if it is .tsv, most likely the sep = "\t"
index_col = 0
header = 0
sep = ','


import os
import re

# Example usage
grn_filenames = (grn_directory+"bonobo_th01_phase-G1_cc-0_sample-0.h5",  grn_directory + "bonobo_th01_phase-S_cc-0_sample-0.h5",grn_directory+"bonobo_th01_phase-G2-M_cc-0_sample-0.h5")



# load the gene regulatory networks into PANDAS dataframes and then to a list
grns = []
for grn_filename in grn_filenames:
    if 'h5' not in grn_filename:
        grn_pd = pd.read_csv(
            grn_filename,  index_col=index_col, header=header, sep=sep)
    else:
        grn_pd = pd.read_hdf(grn_filename)

    grns.append(grn_pd)

In [ ]:
# grns[0].to_hdf("../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_MALE-tumor_power10.h5",key='df')
# grns[1].to_hdf("../data/networks/TCGA_mf_contrast/tcga_kirc_sex/wgcna/wgcna_recount3_tcga_kirc_purity03_normlogtpm_mintpm1_fracsamples01_tissueall_batchnull_adjnull_FEMALE-tumor_power10.h5",key='df')

In [ ]:
#print some stats
col_genes = grns[0].columns.to_numpy()
row_genes = col_genes


total_genes = np.append(row_genes, col_genes)
total_genes = np.unique(total_genes)

num_rows = np.size(row_genes)
num_cols = np.size(col_genes)
num_total = np.size(total_genes)

print(f"There are {num_rows} row genes, {num_cols} column genes, and {num_total} unique genes in first graph")



In [ ]:
# get DeDi
# map gene ids to names if necessary

DeDi = grns[0].sum(axis=0) - grns[1].sum(axis=0)
absDeDi = np.abs(DeDi)

DeDi_genes_id = col_genes

# map ensembl to gene name
# gencode_fn = '../data/gene_set_libraries/human/gen_v26_mapping.csv'
# gencode = pd.read_csv(gencode_fn, index_col=0)
# ens2symbol = {i['gene_id'].split('.')[0]: i['gene_name']
#               for k, i in gencode.iterrows()}
# DeDi_genes_name = [ens2symbol[x[0:15]] for x in DeDi_genes_id]

DeDi_data_dict = {"genes": DeDi_genes_id, "DeDi": DeDi, "absDeDi": absDeDi}

DeDi_data_pd = pd.DataFrame(DeDi_data_dict, index=DeDi_genes_id)
DeDi_data_pd.sort_values(by='absDeDi', ascending=False, inplace=True)

In [ ]:
from node2vec2rank.model import n2v2r

import json



#read the config file
config = json.load(open('../configs/config_bonobo_multi.json', 'r'))

config = {param: value for section, params in config.items()
          for param, value in params.items()}

##the dictionary for mapping indices to gene names
# node_names = DeDi_genes_id
# map orf to gene name
yeast_map_fn = '../data/gene_set_libraries/yeast/yeast_orf_to_symbol_mapping.tsv'
yeast_map = pd.read_csv(yeast_map_fn, sep="\t")
orf2symbol = {i['orf']:i['name'] for k,i in yeast_map.iterrows()}
DeDi_genes_name = [orf2symbol[x] if x in orf2symbol else x for x in DeDi_genes_id ]

graphs_np = [grn.to_numpy() for grn in grns]
model = n2v2r(graphs=graphs_np, config=config, node_names=DeDi_genes_name)
rankings = model.fit_transform_rank()

borda_rankings = model.aggregate_transform()

signed_rankings = model.signed_ranks_transform(DeDi_data_pd.iloc[:,1])

In [ ]:
target = 'sVSg2'


comparison = '2vs3'
n2v2r_ranking_pd = rankings[comparison]
n2v2r_borda_ranking_pd = borda_rankings[comparison]
n2v2r_DeDi_ranking_pd = signed_rankings[comparison]
n2v2r_borda_DeDi_ranking_pd = model.pairwise_signed_aggregate_ranks[comparison]

In [ ]:
# def save_dict_to_csv(dictionary):
#     for key, values in dictionary.items():
#         # Create dataframe
#         df = pd.DataFrame(values, index = row_genes)
        
#         # Save dataframe to CSV
#         filename = f"{key}.csv"
#         df.to_csv(filename, index=True)
#         print(f"Saved {filename}")

# save_dict_to_csv(borda_rankings)


In [ ]:
from node2vec2rank.visualization_utils import dim_reduction, plot_embeddings

algorithm = 'pca'
n_components = 3

node_names = n2v2r_borda_ranking_pd.index.to_list()

first_embeddings = model.node_embeddings[0]
second_embeddings = model.node_embeddings[1]
concat_embeddings = np.append(first_embeddings, second_embeddings, axis=0)

first_embeddings_red = dim_reduction(
    first_embeddings[:, :6], algorithm=algorithm, n_components=n_components)
second_embeddings_red = dim_reduction(
    second_embeddings[:, :6], algorithm=algorithm, n_components=n_components)
concat_embeddings_red = dim_reduction(
    concat_embeddings[:, :6], algorithm=algorithm, n_components=n_components)

plot_embeddings(first_embeddings_red, color_type='numeric',
                color=np.log(n2v2r_borda_ranking_pd.loc[node_names, 'borda_ranks']), names=node_names)
plot_embeddings(second_embeddings_red, color_type='numeric',
                color=np.log(n2v2r_borda_ranking_pd.loc[node_names, 'borda_ranks']), names=node_names)

num_nodes = first_embeddings_red.shape[0]
color_one = np.zeros(num_nodes)
color_two = np.ones(num_nodes)
color_concat = np.append(color_one, color_two, axis=0)

plot_embeddings(concat_embeddings_red, color=color_concat,
                names=np.append(node_names, node_names))

In [ ]:
# run enrich GSEA
from node2vec2rank.post_utils import enrichr_gseapy, read_gmt
from itertools import chain
import os

save_results = True


# read the geneset libraries
kegg_pathway_fn = '../data/gene_set_libraries/yeast/KEGG_2018_yeast.gmt'
gobp_pathway_fn = '../data/gene_set_libraries/yeast/GO_Biological_Process_2018_yeast.gmt'

# network_background or pathway_background for enrichment
# network will use the genes in the network only, while pathway will use all the genes in the pathways
# network is "more fair" but will find less things in small networks
background = 'network_background'
organism = 'yeast'

# take the top k percentage of the ranking for enrichment
top_k_percent = 10

if background == 'network_background':
    kegg_background = n2v2r_ranking_pd.index.to_list()
    gobp_background = n2v2r_ranking_pd.index.to_list()
elif background == 'pathway_background':
    kegg_dict = read_gmt(kegg_pathway_fn)
    kegg_background = list(set(chain.from_iterable(kegg_dict.values())))
    gobp_dict = read_gmt(gobp_pathway_fn)
    gobp_background = list(set(chain.from_iterable(gobp_dict.values())))
else:
    raise Exception("Enrichment background not properly set")

n2v2r_enr_KEGG_pd = enrichr_gseapy(n2v2r_ranking_pd, kegg_pathway_fn, background=kegg_background,enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

n2v2r_enr_GOBP_pd = enrichr_gseapy(n2v2r_ranking_pd, gobp_pathway_fn, background=gobp_background,enrich_quantile_cutoff=top_k_percent/100, organism=organism)

borda_enr_KEGG_pd = enrichr_gseapy(n2v2r_borda_ranking_pd, kegg_pathway_fn, background=kegg_background, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

borda_enr_GOBP_pd = enrichr_gseapy(n2v2r_borda_ranking_pd, gobp_pathway_fn, background=gobp_background, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

absDeDi_enr_KEGG_pd = enrichr_gseapy(DeDi_data_pd[['absDeDi']], kegg_pathway_fn, background=kegg_background, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

absDeDi_enr_GOBP_pd = enrichr_gseapy(DeDi_data_pd[['absDeDi']], gobp_pathway_fn, background=gobp_background, enrich_quantile_cutoff=1-top_k_percent/100, organism=organism)

if save_results:
    path = '../results/results_gsea/' + gsea_parent_folder_name
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    n2v2r_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                       "_n2v2r"+"_consensus_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_enr_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_enr_KEGG_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_enr_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_enr_GOBP_"+background+"_top"+str(top_k_percent)+"_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')


In [ ]:
from node2vec2rank.post_utils import plot_gseapy_enrich 

stability_cutoff = 0.2
padj_cutoff = 0.1

plot_gseapy_enrich(n2v2r_enr_KEGG_pd, show_stability=True, padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff,  title="n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG enrichr " + target)
plot_gseapy_enrich(borda_enr_KEGG_pd, padj_cutoff=padj_cutoff, show_stability=False, stability_cutoff=stability_cutoff, title="Borda n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG enrichr " + target)
plot_gseapy_enrich(absDeDi_enr_KEGG_pd, padj_cutoff=padj_cutoff, show_stability=False, stability_cutoff=stability_cutoff, title="absDeDi " + network_inf_method+ " " +  tissue + " " + "KEGG enrichr " + target)

plot_gseapy_enrich(n2v2r_enr_GOBP_pd, padj_cutoff=padj_cutoff,show_stability=True, stability_cutoff=stability_cutoff,  title="n2v2r " + network_inf_method+ " " +  tissue + " " + "GOBP enrichr " + target)
plot_gseapy_enrich(borda_enr_GOBP_pd, padj_cutoff=padj_cutoff,show_stability=False, stability_cutoff=stability_cutoff, title="Borda n2v2r " + network_inf_method+ " " +  tissue + " " + "GOBP enrichr " + target)
plot_gseapy_enrich(absDeDi_enr_GOBP_pd, padj_cutoff=padj_cutoff, show_stability=False, stability_cutoff=stability_cutoff, title="absDeDi " + network_inf_method+ " " +  tissue + " " + "GOBP enrichr " + target)


In [ ]:
# run prerank GSEA
from node2vec2rank.post_utils import prerank_gseapy

save_results = True

# read the geneset libraries
kegg_pathway_fn = '../data/gene_set_libraries/yeast/KEGG_2018_yeast.gmt'
gobp_pathway_fn = '../data/gene_set_libraries/yeast/GO_Biological_Process_2018_yeast.gmt'

prerank_weight = 0
prerank_min_path_size = 5
prerank_max_path_size = 1500
prerank_num_perms = 1000

n2v2r_pre_KEGG_pd = prerank_gseapy(n2v2r_ranking_pd, kegg_pathway_fn, prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_pre_GOBP_pd = prerank_gseapy(n2v2r_ranking_pd, gobp_pathway_fn,prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

borda_pre_KEGG_pd = prerank_gseapy(n2v2r_borda_ranking_pd, kegg_pathway_fn, prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

borda_pre_GOBP_pd = prerank_gseapy(n2v2r_borda_ranking_pd, gobp_pathway_fn, prerank_weight=prerank_weight,
                                 prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

absDeDi_pre_KEGG_pd = prerank_gseapy(DeDi_data_pd[['absDeDi']], kegg_pathway_fn, prerank_weight=prerank_weight,
                                   prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

absDeDi_pre_GOBP_pd = prerank_gseapy(DeDi_data_pd[['absDeDi']], gobp_pathway_fn, prerank_weight=prerank_weight,
                                   prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

DeDi_pre_KEGG_pd = prerank_gseapy(DeDi_data_pd[['DeDi']], kegg_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

DeDi_pre_GOBP_pd = prerank_gseapy(DeDi_data_pd[['DeDi']], gobp_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_borda_DeDi_pre_KEGG_pd = prerank_gseapy(n2v2r_borda_DeDi_ranking_pd, kegg_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                            prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_borda_DeDi_pre_GOBP_pd = prerank_gseapy(n2v2r_borda_DeDi_ranking_pd, gobp_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                            prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_DeDi_pre_KEGG_pd = prerank_gseapy(n2v2r_DeDi_ranking_pd, kegg_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                      prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

n2v2r_DeDi_pre_GOBP_pd = prerank_gseapy(n2v2r_DeDi_ranking_pd, gobp_pathway_fn, one_sided=False, prerank_weight=prerank_weight,
                                      prerank_min_path_size=prerank_min_path_size, prerank_max_path_size=prerank_max_path_size, prerank_num_perms=prerank_num_perms, num_threads=n_cores)

if save_results:
    path = '../results/results_gsea/' + gsea_parent_folder_name
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    n2v2r_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                             "_n2v2r"+"_consensus_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                             "_n2v2r"+"_consensus_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    borda_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target+"_n2v2r" +
                             "_borda_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    absDeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                               "_absDeDi"+"_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_DeDi"+"_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                            "_DeDi"+"_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_borda_DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                        "_n2v2r_borda_DeDi"+"_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_borda_DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                        "_n2v2r_borda_DeDi"+"_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_DeDi_pre_KEGG_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                  "_n2v2r_chimera"+"_prerank_KEGG_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')
    n2v2r_DeDi_pre_GOBP_pd.to_csv(path+"/"+tissue+"_"+network_inf_method+"_"+target +
                                  "_n2v2r_chimera"+"_prerank_GOBP_"+save_gsea_results_notes+".tsv", header=True, index=None, sep='\t')

In [ ]:
from node2vec2rank.post_utils import plot_gseapy_prerank 

stability_cutoff = 0.2
padj_cutoff = 0.25

plot_gseapy_prerank(n2v2r_pre_KEGG_pd, show_stability=True, padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)
plot_gseapy_prerank(borda_pre_KEGG_pd, show_stability=False,padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="Borda n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)
plot_gseapy_prerank(absDeDi_pre_KEGG_pd,show_stability=False,padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="absDeDi " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)
plot_gseapy_prerank(n2v2r_DeDi_pre_KEGG_pd,show_stability=True,padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="signed n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)
plot_gseapy_prerank(n2v2r_borda_DeDi_pre_KEGG_pd,show_stability=False,padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="signed Borda n2v2r " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)

plot_gseapy_prerank(DeDi_pre_KEGG_pd,show_stability=False,padj_cutoff=padj_cutoff, stability_cutoff=stability_cutoff, title="DeDi " + network_inf_method+ " " +  tissue + " " + "KEGG prerank " + target)
